In [ ]:
import numpy as np
import pandas as pd
import pprint as pp
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
sns.lineplot(x = "date", y = "dogecoin_mention_polarity_score", color='blue', data = elon_doge_df) 
sns.lineplot(x = "date", y = "close",  color='green', data = elon_doge_df) 
plt.ylabel("Polarity Score and Dogecoin Price")
plt.legend(labels=['Tweet Sentiment Polarity Score', 'Dogecoin Day Close Price'])

NameError: name 'sns' is not defined

In [ ]:
sns.lineplot(x = "date", y = "dogecoin_mentions", color='blue', data = elon_doge_df) 
sns.lineplot(x = "date", y = "close",  color='green', data = elon_doge_df) 
plt.ylabel("Mentions and Dogecoin Price")
plt.legend(labels=['Number of Mentions', 'Dogecoin Day Close Price'])